In [1]:
import numpy as np
import torch
import transformers

from life_after_bert import MCDataset, evaluate_encoder_decoder

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "t5-large"
task_type = "oLMpics MLM"
task_name = "Age Comparison"
num_choices = 2

In [3]:
model = transformers.T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, mask_token="<extra_id_0>")  # Set sentinel token as mask_token
static_decoder_input_ids = tokenizer("<pad> <extra_id_0>", add_special_tokens=False, return_tensors="pt").input_ids
dataset = MCDataset.load_data(task_name, num_choices, task_type, tokenizer)

In [4]:
accuracy, (all_answers, all_preds) = evaluate_encoder_decoder(model, dataset, static_decoder_input_ids, device=device)
accuracy

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

0.94